# Assignment 1 - Algorithms and Data Structures Analysis

## Overview

This notebook performs empirical analysis of algorithm performance to verify theoretical Big O complexity predictions. We test two fundamental algorithmic problems with multiple implementations to demonstrate how different approaches scale with input size.

## Summary

This notebook tests and visualizes the performance of:

**UnionFind Algorithms** (4 implementations):
- **Quick Find**: O(1) find, O(N) union - simple but inefficient for many unions
- **Quick Union**: O(N) find, O(1) union - better for sparse operations
- **Weighted Quick Union**: O(log N) find, O(log N) union - balanced approach with tree size tracking
- **Weighted Quick Union with Path Compression**: O(α(N)) amortized - optimal performance
- Tested with varying input sizes (1K-100K elements) and proportional operations

**3Sum Algorithms** (3 implementations):
- **Brute Force**: O(N³) - checks all possible triplets
- **Optimized Two Pointers**: O(N²) - sorts array and uses two-pointer technique
- **Hash Set**: O(N²) - uses hash table for constant-time lookups
- Tested with different array sizes (80-8K elements)

**Analysis Methodology**:
- Performance plots comparing execution times across input sizes
- Log-log scale plots to verify Big O complexity slopes (slope = complexity exponent)
- Statistical timing measurements using %timeit for accuracy
- Complexity verification through slope analysis


In [ ]:
import sys

# Add src directory to path to import custom algorithm implementations
sys.path.append("../src")

import random

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Import custom algorithm implementations
from threesum import (
    generate_test_data,
    three_sum_brute_force,
    three_sum_optimized,
    three_sum_optimized_with_hash,
)
from unionfind import (
    QuickFind,
    QuickUnion,
    WeightedQuickUnion,
    WeightedQuickUnionPathCompression,
)

# Configure plotting style for pretty-looking graphs
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")

# Set random seeds for reproducible results across runs for consistency
random.seed(42)
np.random.seed(42)

print("Libraries imported successfully!")
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")

In [ ]:
# Helper functions for performance testing and data generation


def setup_unionfind_test(uf_class, n: int, operations: list[tuple[int, int]]):
    """
    Setup UnionFind instance and operations for %timeit testing.

    This function creates a closure that contains the UnionFind instance
    and all operations to be performed, allowing %timeit to measure
    just the algorithm execution time without setup overhead.
    """
    uf = uf_class(n)

    def run_operations():
        for p, q in operations:
            uf.union(p, q)

    return run_operations


def generate_unionfind_operations(n: int, num_operations: int) -> list[tuple[int, int]]:
    """
    Generate random union operations for testing.

    Creates a list of random (p, q) pairs where both p and q are
    valid indices in the range [0, n-1]. The number of operations
    is typically proportional to n to ensure meaningful performance
    comparisons across different input sizes.
    """
    operations = []
    for _ in range(num_operations):
        p = random.randint(0, n - 1)
        q = random.randint(0, n - 1)
        operations.append((p, q))
    return operations


def setup_threesum_test(func, nums: list[int]):
    """
    Setup 3Sum function for %timeit testing.

    Creates a closure that contains the test data and function
    to be tested, allowing %timeit to measure just the algorithm
    execution time without data generation overhead.

    Uses value-based deduplication to avoid inflated result counts.
    """

    def run_threesum():
        return func(nums, return_values=True)

    return run_threesum


print("Performance measurement functions defined!")

In [ ]:
# =============================================================================
# UnionFind Performance Analysis
# =============================================================================
# This section tests the performance of four different UnionFind implementations
# across varying input sizes to verify their theoretical time complexities.

print("=== UnionFind Performance Analysis ===")

# Test parameters: Input sizes from 1K to 100K elements
# These sizes are chosen to show clear performance differences between algorithms
# and allow for meaningful slope analysis in log-log plots
n_values = [1000, 5000, 10000, 50000, 100000]

# UnionFind algorithms to test - ordered from least to most efficient
uf_algorithms = [
    ("Quick Find", QuickFind),  # O(1) find, O(N) union
    ("Quick Union", QuickUnion),  # O(N) find, O(1) union
    ("Weighted Quick Union", WeightedQuickUnion),  # O(log N) find, O(log N) union
    (
        "Weighted Quick Union with Path Compression",
        WeightedQuickUnionPathCompression,
    ),  # O(α(N)) amortized
]

# Store performance results for analysis and visualization
uf_results = []

# Test each algorithm with each input size
for n in n_values:
    # Scale operations proportionally to N
    # 0.9 ratio ensures sufficient operations while avoiding complete connectivity
    # This creates a realistic workload where most elements get connected
    num_operations = int(0.9 * n)
    print(f"\nTesting with N = {n}, Operations = {num_operations}")
    operations = generate_unionfind_operations(n, num_operations)

    # Test each UnionFind implementation with the same set of operations
    for name, uf_class in uf_algorithms:
        test_func = setup_unionfind_test(uf_class, n, operations)

        # Use %timeit for accurate timing measurements
        # -q flag suppresses output, -o flag returns timing object
        result = %timeit -q -o test_func() # pyright: ignore

        # Store results for later analysis
        uf_results.append(
            {
                "Algorithm": name,
                "N": n,
                "Operations": num_operations,
                "Time (s)": result.best,  # Best time from multiple runs
                "Average (s)": result.average,  # Average time from multiple runs
            }
        )
        print(f"  {name}: {result.best:.6f}s (best), {result.average:.6f}s (avg)")

print(f"\nUnionFind analysis completed! {len(uf_results)} measurements taken.")

In [ ]:
# =============================================================================
# 3Sum Performance Analysis
# =============================================================================
# This section tests three different approaches to the 3Sum problem:
# 1. Brute Force: O(N³) - checks all possible triplets
# 2. Optimized Two Pointers: O(N²) - sorts array and uses two-pointer technique
# 3. Hash Set: O(N²) - uses hash table for constant-time lookups
#
# NOTE: All algorithms now use value-based deduplication (return_values=True)
# to avoid inflated result counts from duplicate values in random test data.

print("=== 3Sum Performance Analysis ===")

# Test parameters for 3Sum algorithms
# Different array sizes are used for different algorithms based on their complexity:
# - Brute force: smaller sizes (80-400) due to O(N³) complexity
# - Optimized algorithms: larger sizes (500-8000) due to O(N²) complexity
array_sizes_brute = [80, 120, 200, 300, 400]  # Smaller sizes for O(N³) algorithm
array_sizes_optimized = [
    500,
    1000,
    2000,
    5000,
    8000,
]  # Larger sizes for O(N²) algorithms

# 3Sum algorithms to test - ordered by theoretical efficiency
threesum_algorithms = [
    ("Brute Force", three_sum_brute_force),  # O(N³) - checks all triplets
    ("Optimized Two Pointers", three_sum_optimized),  # O(N²) - two-pointer technique
    ("Hash Set", three_sum_optimized_with_hash),  # O(N²) - hash table approach
]

# Store performance results for analysis and visualization
threesum_results = []

# Test brute force algorithm with smaller array sizes
# Due to O(N³) complexity, we use smaller sizes to keep execution time reasonable
print("\n--- Testing Brute Force with smaller sizes ---")
for size in array_sizes_brute:
    print(f"\nTesting Brute Force with array size = {size}")

    # Generate random test data for this array size
    test_data = generate_test_data(size)

    name, func = threesum_algorithms[0]  # Brute Force algorithm
    test_func = setup_threesum_test(func, test_data)

    # Run algorithm once to get solution count and verify correctness
    sample_result = func(test_data, return_values=True)
    result = %timeit -q -o test_func()  # pyright: ignore

    # Store results including number of solutions found
    threesum_results.append(
        {
            "Algorithm": name,
            "Array Size": size,
            "Time (s)": result.best,
            "Average (s)": result.average,
            "Solutions Found": len(sample_result),
        }
    )
    print(
        f"  {name}: {result.best:.6f}s (best), "
        f"{result.average:.6f}s (avg), {len(sample_result)} unique value triplets"
    )

# Test optimized algorithms with larger array sizes
# Due to O(N²) complexity, test with larger sizes and keeping execution time reasonable
print("\n--- Testing Optimized algorithms with larger sizes ---")
for size in array_sizes_optimized:
    print(f"\nTesting with array size = {size}")

    # Generate random test data for this array size
    test_data = generate_test_data(size)

    # Test both optimized algorithms (skip brute force)
    for name, func in threesum_algorithms[1:]:  # Skip brute force
        test_func = setup_threesum_test(func, test_data)

        # Run algorithm once to get solution count and verify correctness
        sample_result = func(test_data, return_values=True)
        result = %timeit -q -o test_func()  # pyright: ignore

        # Store results including number of solutions found
        threesum_results.append(
            {
                "Algorithm": name,
                "Array Size": size,
                "Time (s)": result.best,
                "Average (s)": result.average,
                "Solutions Found": len(sample_result),
            }
        )
        print(
            f"  {name}: {result.best:.6f}s (best), "
            f"{result.average:.6f}s (avg), {len(sample_result)} unique value triplets"
        )

print(f"\n3Sum analysis completed! {len(threesum_results)} measurements taken.")

In [ ]:
# =============================================================================
# Performance Visualization and Complexity Analysis
# =============================================================================
# This section creates visualizations and performs complexity analysis to verify
# that the empirical results match the theoretical Big O predictions.

print("=== Creating Performance Visualizations ===")

# Convert results to DataFrames for easier plotting and analysis
uf_df = pd.DataFrame(uf_results)
threesum_df = pd.DataFrame(threesum_results)

# Slope verification for complexity analysis
# In log-log plots, the slope of the line represents the exponent in the complexity
# For example: O(N²) has slope ≈ 2, O(N³) has slope ≈ 3, O(log N) has slope ≈ 0.3
print("\n=== Slope Analysis for Complexity Verification ===")

# Minimum data points required for meaningful slope calculation
MIN_DATA_POINTS = 2

# UnionFind slope analysis
# Expected slopes: Quick Find ≈ 1 (O(N) union), Quick Union ≈ 1 (O(N) find),
# Weighted Quick Union ≈ 0.3 (O(log N)), Path Compression ≈ 0.1 (O(α(N)))
print("\nUnionFind Complexity Verification:")
for algorithm in uf_df["Algorithm"].unique():
    data = uf_df[uf_df["Algorithm"] == algorithm].sort_values("N")
    if len(data) >= MIN_DATA_POINTS:
        # Calculate slope between first and last points in log-log space
        # This gives us the exponent of the complexity function
        log_n = np.log10(data["N"].values)
        log_time = np.log10(data["Time (s)"].values)
        slope = (log_time[-1] - log_time[0]) / (log_n[-1] - log_n[0])
        print(f"  {algorithm}: slope = {slope:.2f}")

# 3Sum slope analysis
# Expected slopes: Brute Force ≈ 3 (O(N³)), Optimized algorithms ≈ 2 (O(N²))
print("\n3Sum Complexity Verification:")
for algorithm in threesum_df["Algorithm"].unique():
    data = threesum_df[threesum_df["Algorithm"] == algorithm].sort_values("Array Size")
    if len(data) >= MIN_DATA_POINTS:
        # Calculate slope in log-log space to determine complexity exponent
        log_size = np.log10(data["Array Size"].values)
        log_time = np.log10(data["Time (s)"].values)
        slope = (log_time[-1] - log_time[0]) / (log_size[-1] - log_size[0])
        print(
            f"  {algorithm}: slope = {slope:.2f} "
            f"(expected: ~2.0 for O(N²), ~3.0 for O(N³))"
        )

# Create comprehensive performance visualization with 4 subplots
plt.figure(figsize=(12, 8))

# Subplot 1: UnionFind performance on linear scale
# Shows absolute performance differences between algorithms
plt.subplot(2, 2, 1)
for algorithm in uf_df["Algorithm"].unique():
    data = uf_df[uf_df["Algorithm"] == algorithm]
    plt.plot(data["N"], data["Time (s)"], marker="o", label=algorithm)
plt.xlabel("Number of Elements (N)")
plt.ylabel("Time (seconds)")
plt.title("UnionFind Performance Comparison")
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: UnionFind performance on log-log scale
# Log-log plots reveal the complexity by showing straight lines with slopes
# equal to the complexity exponent (e.g., slope 1 = O(N), slope 2 = O(N²))
plt.subplot(2, 2, 2)
for algorithm in uf_df["Algorithm"].unique():
    data = uf_df[uf_df["Algorithm"] == algorithm]
    plt.loglog(data["N"], data["Time (s)"], marker="o", label=algorithm)
plt.xlabel("Number of Elements (N) - Log Scale")
plt.ylabel("Time (seconds) - Log Scale")
plt.title("UnionFind Performance (Log-Log Scale)")
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 3: 3Sum performance on linear scale
# Shows how the O(N³) brute force algorithm becomes impractical for larger inputs
# while the O(N²) optimized algorithms scale much better
plt.subplot(2, 2, 3)
for algorithm in threesum_df["Algorithm"].unique():
    data = threesum_df[threesum_df["Algorithm"] == algorithm]
    plt.plot(data["Array Size"], data["Time (s)"], marker="s", label=algorithm)
plt.xlabel("Array Size")
plt.ylabel("Time (seconds)")
plt.title("3Sum Performance Comparison")
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 4: 3Sum performance on log-log scale
# The slope of the lines reveals the complexity: Brute Force should have slope ≈ 3,
# while optimized algorithms should have slope ≈ 2
plt.subplot(2, 2, 4)
for algorithm in threesum_df["Algorithm"].unique():
    data = threesum_df[threesum_df["Algorithm"] == algorithm]
    plt.loglog(data["Array Size"], data["Time (s)"], marker="s", label=algorithm)
plt.xlabel("Array Size - Log Scale")
plt.ylabel("Time (seconds) - Log Scale")
plt.title("3Sum Performance (Log-Log Scale)")
plt.legend()
plt.grid(True, alpha=0.3)

# Adjust layout to prevent overlapping subplots
plt.tight_layout()
plt.show()

print("Visualizations created successfully!")